In [1]:
from zad11 import connect, load_table, process

In [2]:
spark = connect()

Connecting to Spark...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 17:16:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/10 17:16:26 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
load_table(spark, "/input/daily_weather_2017.csv")

Loading '/input/daily_weather_2017.csv' into table 'daily_weather_2017'...


DataFrame[station_id: string, date: string, avg_temp_c: string, precipitation_mm: string]

In [4]:
load_table(spark, "/input/cities.csv")

Loading '/input/cities.csv' into table 'cities'...


DataFrame[station_id: string, city_name: string, country: string, state: string, iso2: string, iso3: string, latitude: string, longitude: string]

In [5]:
process(spark, "daily_country_weather", """
SELECT
    c.country,
    w.date,
    AVG(w.temperature_c) as temperature_c,
    COALESCE(AVG(w.precipitation_mm), 0) as precipitation_mm
FROM (
    SELECT
        station_id,
        to_date(date) as date,
        avg_temp_c as temperature_c,
        precipitation_mm
    FROM daily_weather_2017
    WHERE date BETWEEN '2017-01-01' AND '2021-12-31'
) AS w
JOIN cities AS c
    ON w.station_id = c.station_id
GROUP BY c.country, w.date
HAVING temperature_c IS NOT NULL
ORDER BY c.country, w.date
""")

Processing query and saving to '/daily_country_weather/*'...


Execution time: 5.779 seconds


+-----------+----------+--------------------+----------------+
|    country|      date|       temperature_c|precipitation_mm|
+-----------+----------+--------------------+----------------+
|Afghanistan|2017-01-01|  5.3166666666666655|             0.0|
|Afghanistan|2017-01-02|   5.016666666666667|             2.0|
|Afghanistan|2017-01-03|  3.0666666666666664|          10.725|
|Afghanistan|2017-01-04|                2.65|           109.0|
|Afghanistan|2017-01-05|  1.9333333333333333|           29.95|
|Afghanistan|2017-01-06|  0.9833333333333331|            7.25|
|Afghanistan|2017-01-07| 0.28333333333333344|            33.0|
|Afghanistan|2017-01-08|-0.07999999999999999|            18.0|
|Afghanistan|2017-01-09|-0.21666666666666676|             0.0|
|Afghanistan|2017-01-10|   0.866666666666667|             0.0|
|Afghanistan|2017-01-11|  0.4499999999999999|             0.0|
|Afghanistan|2017-01-12|  -0.616666666666667|             0.0|
|Afghanistan|2017-01-13| -0.8399999999999999|          

In [6]:
spark.stop()